In [1]:
import pandas as pd
import numpy as np
import scipy 

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from random import choices,sample

import os
import gc
import pickle

import warnings
warnings.filterwarnings('ignore')

from datetime import timedelta
from tqdm import tqdm

plt.style.use('ggplot')

mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = "Malgun Gothic"

os.chdir("../../")
root_path = os.getcwd()

raw_file_path = os.path.join(root_path, "Bigcon2020_raw_file")
csv_file_path = os.path.join(root_path, "BigCon_2020/csv_files")
pickle_file_path = os.path.join(root_path, "BigCon_2020/pickle_files")

raw_file_folders = os.listdir(raw_file_path)

folder_path = os.path.join(raw_file_path, raw_file_folders[0])

In [45]:
seoul_pop = pd.read_table("행정동_가구원수별_가구수.txt", header=[1], thousands=',')
daegu_pop = pd.read_csv("구·군_동·읍·면별_세대원수별_세대수_20200809161016.csv", encoding='cp949', header=[1])

seoul_pop.drop('기간', axis=1, inplace=True)
daegu_pop = daegu_pop.iloc[:,[0,1,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1]]

In [46]:
cj = pd.read_csv(os.path.join(csv_file_path, "2020빅콘_물류_원본.csv"), thousands=',')
cj_cat_merge = cj.groupby(['DL_YMD','CTPV_NM','CTGG_NM','HDNG_NM']).sum().iloc[:,-1].reset_index(name='INVC_CONT')

In [47]:
daegu_pop.columns = ['GU_NM','HDONG_NM','total','pop1','pop2','pop3','pop4','pop5','pop6','pop7','pop8','pop9']
seoul_pop.columns = ['GU_NM','HDONG_NM','total','pop1','pop2','pop3','pop4','pop5','pop6','pop7']

In [48]:
total_dong = flow_time.HDONG_NM.unique().tolist()

total_gu = ['중구','수성구','노원구']
total_dong += ['수성2·3가동']

In [49]:
daegu_pop = daegu_pop.loc[(daegu_pop.HDONG_NM.isin(total_dong)) & (daegu_pop.GU_NM.isin(total_gu))].reset_index(drop=True)
seoul_pop = seoul_pop.loc[(seoul_pop.HDONG_NM.isin(total_dong)) & (seoul_pop.GU_NM.isin(total_gu))].reset_index(drop=True)

In [50]:
cj_cat_merge.rename(columns={'CTGG_NM':'GU_NM', 'HDNG_NM':'HDONG_NM'}, inplace=True)

In [51]:
cj_cat_merge=cj_cat_merge.replace('수성2·3가동', '수성2.3가동')
cj_cat_merge=cj_cat_merge.replace('신당제5동', '신당5동')

In [25]:
"""
cj_cat_merge['DL_YMD'] = pd.to_datetime(cj_cat_merge['DL_YMD'].apply(lambda x:'20'+str(x)), format='%Y%m%d')

cj_cat_merge['dayofweek'] = cj_cat_merge.DL_YMD.dt.dayofweek.values
cj_cat_merge['weekday'] = np.where(cj_cat_merge.dayofweek < 5, '평일', '주말')

cj_cat_merge = cj_cat_merge.loc[cj_cat_merge.weekday=='평일'].reset_index()
"""

In [52]:
daegu_pop=daegu_pop.replace('수성2·3가동', '수성2.3가동')

In [53]:
tmp1=cj_cat_merge.merge(daegu_pop)
tmp2=cj_cat_merge.merge(seoul_pop)

In [54]:
tmp1=tmp1.iloc[:,:-2]

In [55]:
cj_with_pop = pd.concat([tmp1,tmp2]).reset_index(drop=True)

In [57]:
cj_with_pop.iloc[:,4:].corr().iloc[:1,2:]

,pop1,pop2,pop3,pop4,pop5
INVC_CONT,0.350237,0.378518,0.395081,0.351328,0.33915


주말포함

In [44]:
cj_with_pop.iloc[:,4:].corr().iloc[:1,3:]

,pop1,pop2,pop3,pop4,pop5
INVC_CONT,0.502524,0.54079,0.563414,0.502939,0.486082


주말제외

In [20]:
cj_with_pop.head(20)

,DL_YMD,CTPV_NM,GU_NM,HDONG_NM,INVC_CONT,total,pop1,pop2,pop3,pop4,pop5,pop6,pop7
0,190201,대구광역시,수성구,고산1동,333,1862,2310,2775,3192,743,116,28,8
1,190202,대구광역시,수성구,고산1동,56,1862,2310,2775,3192,743,116,28,8
2,190203,대구광역시,수성구,고산1동,10,1862,2310,2775,3192,743,116,28,8
3,190204,대구광역시,수성구,고산1동,2,1862,2310,2775,3192,743,116,28,8
4,190205,대구광역시,수성구,고산1동,1,1862,2310,2775,3192,743,116,28,8
5,190206,대구광역시,수성구,고산1동,948,1862,2310,2775,3192,743,116,28,8
6,190207,대구광역시,수성구,고산1동,3689,1862,2310,2775,3192,743,116,28,8
7,190208,대구광역시,수성구,고산1동,2329,1862,2310,2775,3192,743,116,28,8
8,190209,대구광역시,수성구,고산1동,479,1862,2310,2775,3192,743,116,28,8
9,190210,대구광역시,수성구,고산1동,224,1862,2310,2775,3192,743,116,28,8
